# Evaluating Numerical Expressions

## Introduction

In this project, we will use stacks to implement an algorithm that can evaluate complex numerical expressions that are stored in string format (e.g. "12 / ( 2 + 4 ) * 21"). We will use the stack data structure to schedule our processes.

Before we start describing and implementing the algorithm, let's implement a stack data structure class. This will incorporate the LinkedList class, which we will import from the `linked_list.py` script.

In [1]:
# Implement the Stack scheduling algorithm
from linked_list import LinkedList

class Stack(LinkedList):
    
    def push(self, data): # Add node onto top of stack
        self.append(data)
        
    def peek(self): # Return the top node of the stack
        return self.tail.data
    
    def pop(self): # Return the top node of the stack and remove it
        ret = self.tail.data
        
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        
        return ret

## Implement the Tokenise Function

We are going to implement our evaluation algorithm so that it evaluates expressions written in postfix notation, where the operands appear before the operator (e.g 1 2 +). This makes it much easier for the computer to process.

The first step in doing this is to implement a function `tokenise()` that will tokenise the postfix numerical expression, that is to return a list of the separate elements.
For simplicity, we will assume that the postfix expression string contains spaces between any two tokens.

In [2]:
# Create a tokenise() function
def tokenise(expression):
    return expression.split()

# Test the implementation
expression = "12 2 4 + / 21 *"

print(tokenise(expression))

['12', '2', '4', '+', '/', '21', '*']


## Processing an Operator

To evaluate an expression in postfix notation using a stack, we must read the expression from left to right and do the following:

1. If we find a number, then we push that number to the top of the stack.
2. If we find an operator, we pop the top two elements of the stack, perform the operation, and then push the result to the top of the stack.

However, when we find an operator and pop the top two elements of the stack, it is important that we ensure we operate those two numbers in the correct order (i.e the operations `1 - 2` and `2 - 1` produce different results).
Therefore, the element at the top of the stack is actually the second operand in the expression, while the element that is second to top is the first operand.

Let's write functions to correctly process the `-`, `+`, `*`, `/` and `**` operators.

In [3]:
# A function to process the - token
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)
    
# A function to process the + token
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

# A function to process the * token
def process_times(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)
    
# A function to process the / token
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)
    
# A function to process the ** token
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

## Evaluating Postfix Expressions

We can now implement an algorithm to evaluate an expression in postfix notation. To do so, we need to do the following:

1. Tokenise the expression using the `tokenise()` function.
2. Initialise an empty stack.
3. For each token, do the following:
   1. If the token is an operator, call the corresponding function to process it. For example, if we find a `+`, we call the `process_plus()` function.
   2. Otherwise (the token is a number) we push that number to the top of the stack. Since each token is in a string, we'll need to convert it to a `float` first.
4. Return the value that is left in the stack.


   

In [4]:
# Create a function to evaluate a postfix numerical expression
def evaluate_postfix(expression):
    tokens = tokenise(expression)
    stack = Stack()
    
    for token in tokens:
        if token == "+":
            process_plus(stack)
        elif token == "-":
            process_minus(stack)
        elif token == "*":
            process_times(stack)
        elif token == "/":
            process_divide(stack)
        elif token == "**":
            process_pow(stack)
        else:
            stack.push(float(token))
    
    return stack.pop()

# Test the implementation
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation

We can now evaluate postfix expressions. However, to make this project more useful, we need to enable our algorithm to evaluate expressions in infix notation. This is how notation is most commonly provided by users, and it is unlikely that they will be familiar with postfix.

The first step in doing this is to implement a way of telling the algorithm which operators have the highest precedence within a given expression. In mathematical expressions, `**` operators are evaluated first, followed by `*` and `/`, and then lastly `+` and `-`.

We will use numbers to represent the precedence of the operators; the higher the precedence, the higher the number. We will store this information in a dictionary so that it can be made easily accessible to a Shunting-yard algorithm that we will define later.

In [5]:
# Define a precedence dictionary
precedence = {
    '+': 1,
    '-': 1,
    '*': 2,
    '/': 2,
    '**': 3
    }

# Test the implementation
print(precedence['/'] < precedence['-'])
print(precedence['**'] > precedence['*'])
print(precedence['+'] == precedence['-'])

False
True
True


## Converting Infix to Postfix

Next, we will implement a function `infix_to_postfix()` that, given a string with an expression in infix notation, outputs a string with that expression written in postfix notation.

This function will implement the Shunting-yard algorithm. It starts by tokenizing the infix expression, and then it prosesses the tokens one by one using a stack. It builds the postfix expression by keeping track of a list named `postfix`, which will contain the list of tokens in postfix order.

As this algorithm has quite a few parts, we'll split the implementation into several smaller functions that are responsible for handling one type of token:


* Opening parenthesis tokens, `(`:
  1. Push the token into the stack for later use when we find a closing parenthesis.

In [6]:
def process_opening_parenthesis(stack):
    stack.push('(')

* Closing parenthesis tokens, `)`:
  1. While the top of the stack isn't an opening parenthesis, pop the top element and append it to the postfix token list.
  2. Pop the opening parentheses out of the stack at the end.

In [7]:
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != '(': # While the top of the stack isn't an opening parenthesis
        top = stack.pop()
        postfix.append(top)
    stack.pop() # This will be the opening parenthesis and shouldn't be stored anywhere

* Operator tokens, `+`, `-`, `*`, `/`, or `**`:
  1. While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element and append it to the `postfix` token list.
  2. Push the current operator to the top of the stack.

In [8]:
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and precedence[stack.peek()] >= precedence[operator]:
        top = stack.pop()
        postfix.append(top)
    stack.push(operator) # When the top of the stack is no longer an operator with a greater or equal precedence, push the current operator to the top of the stack

* Operand (any number):
  1. Append the token into the postfix token list.

In [9]:
def process_number(postfix, number):
    postfix.append(number)

## Implementing the Shunting-yard Algorithm

We now have all of the pieces we need to implement the `infix_to_postfix()` function that converts an expression from infix notation to postfix notation.

This function will work as follows:

1. We start by splitting the expression into tokens using the `tokenise()` function.
2. We initialise an empty stack.
3. We intitialise an empty postfix token list.
4. Iterate over all tokens, and for each, call the appropriate function that we defined above.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the `str.join()` method to convert the postfix token list into a string.

In [11]:
# Define a function to convert infix expressions into postfix expressions
def infix_to_postfix(expression):
    tokens = tokenise(expression)
    stack = Stack()
    postfix = []
    
    for token in tokens:
        if token == '(':
            process_opening_parenthesis(stack)
        elif token == ')':
            process_closing_parenthesis(stack, postfix)
        elif token in ('+', '-', '*', '/', '**'):
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    
    while stack.length > 0:
        token = stack.pop()
        postfix.append(token)
    
    return ' '.join(postfix)



## Evaluating Infix Expressions

Let's combine the `infix_to_postfix()` and `evaluate_postfix()` functions to evaluate strings containing numerical expressions written in infix notation.

In [13]:
# Define a function that evaluates infix numerical expressions
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

# Test the implementation
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

for expression in expressions:
    print(evaluate(expression))
    

2.0
0.0
8.0
11.25
256.0
65536.0
0.5
9.0
1.0
